## Домашнее задание # 1: Sentiment Analysis при помощи словарей и множеств


> Сначала нам надо скачать дату
>>соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше)
>> 10 отзывов для проверки качества.   
>2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга


In [1]:
import requests
from bs4 import BeautifulSoup

Я не очень долго думала, какой сайт хочу парсить - сердечко просто шепнуло *irecommend*.

Жалею ли я - вопрос интересный. Скорее нет, чем да, потому что работа с таким *хорошо защищенным от DDoS атак* сайтом дала мне богатый опыт и позволила лишний раз поупражнять тепрение и выдержку.

Алгоритм парсинга классический:
* соскрейпить ссылки на достаточное количество языкового материала
* последовательно этот материал собрать

Имея душевные силы и желая избежать хардкодинг любой ценой, я решила, что не буду смотреть глазами и прописывать руками максимально возможную страницу. Не то чтобы у меня был шанс ее случайно достингнуть, но тогда я об этом еще не знала. 

В ретроспективе, конечно, выглядит забавно.

In [3]:
page = 0
max_page = 1

review_hrefs = []

while page != max_page:
    url = f'https://irecommend.ru/content/ottenochnyi-balzam-tonika?page={page}'
    response = requests.get(url)
    if response.status_code != 200:
        print('ERROR code', response.status_code)
        break
    html = BeautifulSoup(response.text, 'html.parser')
    if page == 0:
        max_page = int(html.find_all('li', 
                                        {'class': 'pager-last last'})[0].text)
    reviews = html.find_all('div', 
                            {'class': 'product-node'})[0].find_all('div', 
                                                                   {'class': 
                                                                    'reviewTitle'})
    for review in reviews:
        review_hrefs.append('https://irecommend.ru' + review.find('a').get('href'))
    page += 1

ERROR status code 521


In [ ]:
print(f'Анализируемые в настоящей работе отзывы посвящены \
товару с названием "{html.title.string}"')

Анализируемые в настоящей работе отзывы посвящены товару с названием "Оттеночный бальзам для волос Тоника РоКОЛОР | Отзывы покупателей"


In [105]:
print(f'Before getting banned we managed to collect {len(review_hrefs)} full review hrefs')

Before getting banned we managed to collect 450 full review hrefs


Сбор языкового материала заключается не только в сохранении текстов, но и в запоминании их оценок. 

Сохраняем тексты в файлы с соответствующими названиями, а оценки - в словарь с ключами-индексами.

In [41]:
review_ratings = {}

In [ ]:
for index in range(len(review_hrefs)):
    response = requests.get(review_hrefs[index], 
                            headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    # sleep(randint(1, 5))
    if not response.status_code == 200:
        print('ERROR status code', response.status_code)
        break
    html = BeautifulSoup(response.text, 'html.parser')
    with open(f'text_{index}.txt', 'w', encoding='utf-8') as file:
        file.write(html.find('div', {'class': 'views-field-teaser reviewText'}).text)
    grade = int(html.find('meta', {'itemprop': 'ratingValue'}).get('content'))
    review_ratings[index] = grade

ERROR status code 521


In [ ]:
print(f'overall we have {len(review_ratings)} grades saved')
print(f'{len([i for i in review_ratings if review_ratings[i] > 3])} \
of which are 4 or 5')
print(f'and {len([i for i in review_ratings if review_ratings[i] < 3])}\
 of which are below 3')

overall we have 118 grades saved
64 of which are 4 or 5
and 38 of which are below 3


Немного несбалансированно получилось, но мы попробуем сделать на это поправку в будущем. Строго говоря, того что я собрала - достаточно. 

Формируем обучающую и валидационную выборки. Не руками, конечно. 

А еще для чистоты эксперимента не рассматриваем отзывы без однозначной оценки: классификация у нас бинарная.

In [43]:
from random import sample

negative_test = sample([i for i in review_ratings if review_ratings[i] < 3], 5)
positive_test = sample([i for i in review_ratings if review_ratings[i] > 3], 5)

negative_train = [i for i in review_ratings if review_ratings[i] <= 3 
                  and i not in negative_test]
postitive_train = [i for i in review_ratings if review_ratings[i] > 3 
                   and i not in positive_test]

> Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

In [ ]:
def create_text_data(indices):
    text = ''
    for index in indices:
        with open(f'review_texts/text_{index}.txt', 'r', encoding='utf-8') as file:
            text += '\n' + file.read()
    return text

negative_text = create_text_data(negative_train)
positive_text = create_text_data(postitive_train)

In [5]:
! pip install spacy==3.0.0 -q
! python -m spacy download ru_core_news_sm -q

     |████████████████████████████████| 12.7 MB 214 kB/s 
     |████████████████████████████████| 9.1 MB 25.4 MB/s 
     |████████████████████████████████| 623 kB 30.4 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 456 kB 52.0 MB/s 
     |████████████████████████████████| 17.9 MB 158 kB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 8.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [6]:
%%time

import spacy
nlp = spacy.load('ru_core_news_sm')

def preprocess(text):
    clean = ''.join([i.lower() for i in text
                     if i.isalpha() or i.isspace()]) 
    tokens = [token.lemma_ for token in nlp(clean) if not token.is_stop 
          and token.lemma_.isalpha()] 
    return tokens

preprocessed_positive = preprocess(positive_text)
preprocessed_negative = preprocess(negative_text)

CPU times: user 1min 2s, sys: 2.1 s, total: 1min 4s
Wall time: 1min 4s


> Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

Я надеюсь это не очень критично, что я не обратилась к типу данных *множество*, он не показался мне удобным в данном случае: словарные ключи и так уникальны, но к ним еще и можно намертво прикрепить релевантную информацию о частотности.

In [7]:
def create_frequency_dict(tokens):
    freqs = {}
    for token in tokens:
        if token not in freqs.keys():
            freqs[token] = 0
        freqs[token] += 1
    return freqs

freqs_positive = create_frequency_dict(preprocessed_positive)
freqs_negative = create_frequency_dict(preprocessed_negative)

In [8]:
print(f'Negative reviews contain {len(freqs_negative)} unique tokens, whilst positive ones - {len(freqs_positive)}')

Negative reviews contain 3639 unique tokens, whilst positive ones - 8396


In [15]:
print('negative most popular tokens')
print(*[f'{i[0]} ({i[1]})' for i in sorted(freqs_negative.items(), 
                                           key=lambda x: x[1], 
                                           reverse=True)[:10]])
print('positive most popular tokens')
print(*[f'{i[0]} ({i[1]})' for i in sorted(freqs_positive.items(), 
                                           key=lambda x: x[1], 
                                           reverse=True)[:10]])

negative most popular tokens
волос (505) цвет (214) бальзам (211) тоника (160) оттенок (157) тоник (138) оттеночный (117) очень (102) после (96) результат (69)
positive most popular tokens
волос (1866) оттенок (792) цвет (769) тоника (728) бальзам (555) после (399) тоник (348) очень (339) раз (222) результат (211)


Как и ожидалось, самые абсолютно популярные слова не сильно отличаются для плохих и хороших отзывов: в конце концов эти тексты об одном и том же. 

In [16]:
freq_neg_specific = {key: value for key, value in freqs_negative.items() 
                     if key not in freqs_positive}

freq_pos_specific = {key: value for key, value in freqs_positive.items() 
                     if key not in freqs_negative}

In [17]:
print(f'Negative reviews contain {len(freq_neg_specific)} unique specific tokens, whilst positive ones - {len(freq_pos_specific)}')

Negative reviews contain 1419 unique specific tokens, whilst positive ones - 6176


In [18]:
print('negative most popular specific tokens')
print(*[f'{i[0]} ({i[1]})' for i in sorted(freq_neg_specific.items(), 
                                           key=lambda x: x[1], 
                                           reverse=True)[:10]])
print('positive most popular specific tokens')
print(*[f'{i[0]} ({i[1]})' for i in sorted(freq_pos_specific.items(), 
                                           key=lambda x: x[1], 
                                           reverse=True)[:10]])

negative most popular specific tokens
bloody (5) боль (4) mary (4) огорчить (4) требоваться (4) сомнительный (3) биг (3) пятнистый (3) lux (3) психануть (3)
positive most popular specific tokens
тонировка (26) мешать (21) менять (19) соотношение (18) го (17) устраивать (16) дальше (16) лично (16) подруга (16) предварительно (15)


Вот тут уже интереснее получается. 

Я все-таки хочу убрать слова, встречающиеся единожды, поскольку нередко такие слова - это просто фейлы токенизации, то есть какие-то окказиональные сочетания символов, не то чтобы специфичные для задачи. 

Также я нормализую встречаемость, превращая ее в вес. Это необходимо сделать из-за дисбаланса классов, ибо условная встречаемость X для фичи негативной тональности будет значить намного больше, чем такая же встречаемость Х для позитивной. 

Таким образом, каждая специфичная фича будет давать от 0 до 100 баллов отзыву в направлении той или иной тональности. 

In [22]:
print(f'number of words that occured just once\nnegative: \
{len([i for i in freq_neg_specific if freq_neg_specific[i] == 1])}\
\npositive: {len([i for i in freq_pos_specific if freq_pos_specific[i] == 1])}')

number of words that occured just once
negative: 1325
positive: 4861


In [60]:
neg_features = [i for i in freq_neg_specific.items() if not i[1] == 1]
pos_features = [i for i in freq_pos_specific.items() if not i[1] == 1]

def normalize(features):
    x_max = max([i[1] for i in features])
    return  [(i[0], int(100 * i[1] / x_max)) for i in features]

neg_features_weighted = {i[0]: i[1] 
                         for i in normalize(neg_features) if i[1] != 0}
pos_features_weighted = {i[0]: i[1] 
                         for i in normalize(pos_features) if i[1] != 0}

>Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

Cформулируем коэффициент поправки на разное количество фичей: позитивных слов у нас в разы больше, чем негативных, а это значит, что вероятность появления какой угодно позитивной фичи намного выше, чем вероятность появления какой угодно негативной.


Поэтому мы будем умножать позитивные баллы на отношение количества негативных фичей к позитивным, чтобы привести их примерно к одной шкале.

In [74]:
q = len(neg_features_weighted) / len(pos_features_weighted)

In [81]:
from random import choice

def detect_tonality(text):
    tokens = preprocess(text)
    positive_points = 0
    negative_points = 0
    for token in tokens:
        if token in pos_features_weighted:
            positive_points += pos_features_weighted[token]
        elif token in neg_features_weighted:
            negative_points += neg_features_weighted[token]
    if positive_points * q == negative_points:
        return choice(['positive', 'negative'])
    return 'positive' if positive_points * q > negative_points else 'negative'


In [103]:
test_x = []
for collection in [negative_test, positive_test]:
    for index in collection:
        with open(f'/content/text_{index}.txt', encoding='utf-8') as file:
            test_x.append(file.read())

test_y = ['negative'] * 5 + ['positive'] * 5

def estimate_accuracy(test_x, test_y, model):
    if not len(test_x) == len(test_y):
        raise ValueError
    preds = list(map(model, test_x))
    return len([True for i in zip(preds, test_y) if i[0] == i[1]]) / len(test_y)

print(estimate_accuracy(test_x, test_y, detect_tonality))

0.7


Ну, звучит не так плохо, хотя по факту все не очень: внимательный анализ показал, что выраженный дисбаланс классов привел к тому, что модель с большей охотой относит все подряд к положительным отзывам. И я не могу ее за это осуждать. 

>Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)


1. можно было бы обучать модель не на ~60 положительных и ~30 негативных отзывах, а на ~30 и ~30. Интересно, уравновесит ли выигрыш из-за лучшей сбаланированности потери от меньшего объема. Мне кажется, что вполне может. 

2. а может можно ввести порог: не добавлять баллы за встреченные фичи в разные счетчики, а просто прибавлять позитивные баллы и отнимать негативные, при этом раздел между тем, что в итоге квалифицируется как негативное, и тем, что мы будем считать позитивным, поставить не 0, а что-нибудь повыше. для определения оптимального порога построить AUC-ROC. эта же метрика покажет нам, насколько вообще наш примитивный подход к классификации хорошо ранжирует. 

2. можно было бы обратиться к уже [существующим тональным словарям](https://habr.com/ru/post/482052/) и как-то инкорпорировать эту информацию к информации о специфичных токенов для конкретно нашего материала

> Логичность и чистота кода (1 балл)

In [106]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/jSKikMqhCFo37IezKu/giphy.gif')